In [25]:
import torch
import torchvision
import numpy as np

In [3]:
torch.cuda.is_available()

True

In [4]:
imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=args.nThreads)

NameError: name 'torchvision' is not defined

In [1]:
data = torchvision.datasets.MNIST(root="data/mnist")

NameError: name 'torchvision' is not defined

In [2]:
data = torchvision.datasets.FashionMNIST(root="data/fashion")

NameError: name 'torchvision' is not defined

In [20]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args['dry_run']:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))




args = {'batch-size':64, 'test-batch-size':1000,'epochs':10,'lr':1.0,'gamma':0.7,'no_cuda':False,
       'dry_run':False,'seed':1,'log_interval':10,'save_model':False}

torch.manual_seed(1)

device = torch.device("cuda" if True else "cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}
if True:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    ])
dataset1 = datasets.FashionMNIST('data', train=True,
                   transform=transform)
dataset2 = datasets.FashionMNIST('data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)





In [21]:
%%time
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args['lr'])

scheduler = StepLR(optimizer, step_size=1, gamma=args['gamma'])
for epoch in range(1, args['epochs'] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307203
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.688292
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.673384
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.131498
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.819418
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.836570
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.867201
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.722489
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.905597
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.709511
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.826773
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.710434
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.756560
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.758102
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.769671
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.671683
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.625999
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.531980
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.676744
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.657934
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.389780
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.287123
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.375721
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.173594
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.264314
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.214671
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.193933
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.482786
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.333350
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.364281
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.378740
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.537648
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.277834
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.331855
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.280865
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.311742
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.338227
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.435796
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.260598
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.441761


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.421128
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.194819
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.208498
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.354413
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.206965
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.116020
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.158709
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.258095
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.146301
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.200391
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.182665
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.146585
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.271899
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.308907
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.262788
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.231374
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.288570
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.339024
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.068971
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.388043


Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.115413
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.265496
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.148179
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.240156
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.263638
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.191864
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.258236
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.257294
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.207799
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.331758
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.354791
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.193041
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.112500
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.186590
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.532800
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.180835
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.261182
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.554550
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.180252
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.084885


Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.444506
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.168100
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.207491
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.234914
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.290016
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.338279
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.200368
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.351867

Test set: Average loss: 0.2277, Accuracy: 9191/10000 (92%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.248164
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.079850
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.177855
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.194202
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.206306
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.325227
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.268423
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.171853
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.254709
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.227256
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.109724
Train Epoch: 

Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.130322
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.154931
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.277515
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.123864
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.173169
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.312886
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.131144
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.120189
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.314881
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.313371
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.103898
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.302244
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.143866
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.390897
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.137153
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.143596
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.160884
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.124887
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.389003
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.210557


In [26]:
sum([np.prod(p.size()) for p in model.parameters()])

1199882

In [3]:
%%time
args

Wall time: 0 ns


{'batch-size': 64,
 'test-batch-size': 1000,
 'epochs': 3,
 'lr': 1.0,
 'gamma': 0.7,
 'no_cuda': False,
 'dry_run': False,
 'seed': 1,
 'log_interval': 10,
 'save_model': False}

In [24]:
args['batch-size']

64